In [1]:
from utils import *
import numpy as np
import unidecode

In [2]:
db, engine, meta = db_connect()

In [3]:
tables = db_tables(meta)

In [4]:
leg_map = get_leg_map(engine)

In [5]:
list(leg_map.values())[0]["congress_map"]

{'110': {'chamber': 'senate',
  'congress': 110,
  'bills_sponsored': 15,
  'total_cosponsors': 28,
  'bills_cosponsored': 61,
  'cosponsors_per_bill': 1.8666666666666667,
  'cosponsored_sponsored_ratio': 4.066666666666666,
  'z_bills_sponsored': -0.833809634522492,
  'z_bills_cosponsored': -1.3986844463050083,
  'z_cosponsors_per_bill': -1.1806980542049446,
  'z_cosponsored_sponsored_ratio': -0.6065545631607324,
  'experience': 0,
  'committee_rank_recips': 0.13392857142857142,
  'committee_min_rank': 14.0,
  'committee_count': 4,
  'max_coeff': -0.05,
  'leadership': 0},
 '111': {'chamber': 'senate',
  'congress': 111,
  'bills_sponsored': 16,
  'total_cosponsors': 31,
  'bills_cosponsored': 88,
  'cosponsors_per_bill': 1.9375,
  'cosponsored_sponsored_ratio': 5.5,
  'z_bills_sponsored': -0.7390674214923396,
  'z_bills_cosponsored': -0.7832512437234525,
  'z_cosponsors_per_bill': -0.5368520518913603,
  'z_cosponsored_sponsored_ratio': -0.11193706915066166,
  'experience': 1,
  'commi

In [ ]:
house_assignments = get_csv("data/house_assignments_103-115.csv")
for assignment in house_assignments:
    assignment["chamber"] = "house"

In [ ]:
senate_assignments = get_csv("data/senate_assignments_103-115.csv")
for assignment in senate_assignments:
    assignment["chamber"] = "senate"

In [ ]:
for assignment in house_assignments + senate_assignments:
    assignment["congress"] = assignment.pop("\ufeffcongress")

In [ ]:
def find_by_name(leg_map, last_name, first_name):
    def clean_name(s):
        return unidecode.unidecode(s.strip().lower().replace(" ", "").replace("-", ""))
    last_name = clean_name(last_name)
    first_name = clean_name(first_name)
    cur_filter = [leg for leg in leg_map.values() if clean_name(leg["name"]["last"]) == last_name]
    if len(cur_filter) == 1:
        return cur_filter[0]
    if len(cur_filter) == 0:
        cur_filter = [leg for leg in leg_map.values() if clean_name(leg["name"]["last"]) in last_name or
                            last_name in clean_name(leg["name"]["last"])]
    if len(cur_filter) > 1:
        first_filter_strict = [leg for leg in cur_filter if first_name == clean_name(leg["name"]["first"])]
        if len(first_filter_strict) > 0:
            cur_filter = first_filter_strict
            
    if len(cur_filter) == 1:
        return cur_filter[0]
    else:
        # return cur_filter
        print(len(cur_filter))
        raise Exception("Unable to find {} {}".format(first_name, last_name))

In [ ]:
for assignment in house_assignments + senate_assignments:
    try: 
        if not isinstance(assignment["bioguide"], str) or len(assignment["bioguide"].strip()) == 0:
                names = assignment["Name"].split(", ")
                last_name = names[0]
                first_name = names[1]
                if(first_name[-1] == "."):
                    first_name = first_name[:-2]
                assignment["bioguide"] = find_by_name(leg_map, last_name, first_name)["id"]["bioguide"]
    except:
        print(assignment["Name"])
        # print(assignment)
        # raise Exception()

In [ ]:
def add_bioguide(assignments, name, bioguide):
    for assignment in assignments:
        if assignment["Name"] == name:
            assignment["bioguide"] = bioguide

In [ ]:
find_by_name(leg_map, "", "Jenniffer")

In [ ]:
add_bioguide(senate_assignments + house_assignments, "González, Jenniffer", 'G000582')

In [ ]:
list(range(10))[:-2]

In [ ]:
leg_map["R000600"]

In [ ]:
save_csv(house_assignments, "data/house_assignments_103-115.csv")

In [ ]:
save_csv(senate_assignments, "data/senate_assignments_103-115.csv")

In [ ]:
for leg in leg_map.values():
    leg["committees"] = {}
for assignment in house_assignments + senate_assignments:
    if len(assignment["bioguide"]) == 0:
        continue
    leg = leg_map[assignment["bioguide"]]
    if "committees" not in leg:
        leg["committees"] = {}
    committees = leg["committees"]
    if "congress" not in assignment:
        raise Exception(assignment)
    congress = assignment["congress"]
    if congress not in committees:
        committees[congress] = []
    assignment["committee_name"] = assignment["committee_name"].strip().lower()
    
    committees[congress].append(assignment)

In [ ]:
list(leg_map.values())[0]["committees"]

In [ ]:
for leg in leg_map.values():
    for congress in leg["congress_map"].values():
        congress["leadership"] = 0

In [ ]:
for assignment in senate_assignments + house_assignments:
    if "minority" in assignment["committee_name"] or "majority" in assignment["committee_name"] or "speaker" in assignment["committee_name"]:
        if str(assignment["congress"]) not in leg_map[assignment["bioguide"]]["congress_map"]:
            print(assignment)
            continue
        leg_map[assignment["bioguide"]]["congress_map"][assignment["congress"]]["leadership"] = 1

In [ ]:
leg_map['F000268']["congress_map"]["103"]

In [ ]:
senate_mappings = get_json("data/senate_committee_mapping.json")
house_mappings = get_json("data/house_committee_mapping.json")

In [ ]:
senate_committee_values = get_csv("data/senate_committee_values.csv")
house_committee_values = get_csv("data/house_committee_values.csv")

In [ ]:
senate_values_map = {}
house_values_map = {}
for committee in senate_committee_values + house_committee_values: 
    for key, value in committee.items():
        if key != "committee" and len(value) > 0:
            committee[key] = float(value)
        elif len(value) == 0:
            committee[key] = None
for committee in senate_committee_values:
    senate_values_map[committee["committee"]] = committee
for committee in house_committee_values:
    house_values_map[committee["committee"]] = committee

In [ ]:
save_json(senate_values_map, "data/senate_values_map.json")
save_json(house_values_map, "data/house_values_map.json")

In [ ]:
for leg in leg_map.values():
    congress_map = leg["congress_map"]
    if "committees" in leg:
        for congress_key, congress_val in leg["committees"].items():
            for assignment in congress_val:
                if assignment["chamber"] == "senate":
                    if assignment["committee_name"] in senate_mappings:
                        committee_value = senate_values_map[senate_mappings[assignment["committee_name"]]]
                        assignment["committee_coeff"] = committee_value["coeff"]
                        assignment["committee_rank"] = committee_value["rank"]
                else:
                    if assignment["committee_name"] in house_mappings:
                        committee_value = house_values_map[house_mappings[assignment["committee_name"]]]
                        assignment["committee_coeff"] = committee_value["coeff"]
                        assignment["committee_rank"] = committee_value["rank"]

In [ ]:
list(leg_map.values())[0]["committees"]

In [ ]:
def rank_recip_sums(assignments):
    return sum([1 / assignment["committee_rank"] for assignment in assignments if "committee_rank" in assignment and assignment["committee_rank"] is not None])

In [ ]:
def min_rank(assignments):
    ranks = [assignment["committee_rank"] for assignment in assignments if "committee_rank" in assignment and assignment["committee_rank"] is not None]
    if len(ranks) == 0:
        return None
    else:
        return min(ranks)
    

In [ ]:
def max_coeff(assignments):
    coeffs = [assignment["committee_coeff"] for assignment in assignments if "committee_coeff" in assignment and assignment["committee_coeff"] is not None]
    if len(coeffs) == 0:
        return None
    else:
        return max(coeffs)
    
    

In [ ]:
for leg in leg_map.values():
    committee_map = leg["committees"]
    for congress in leg["congress_map"].values():
        str_congress = str(congress["congress"])
        if str_congress in committee_map:
            congress["committee_rank_recips"] = rank_recip_sums(committee_map[str_congress])
            congress["committee_min_rank"] = min_rank(committee_map[str_congress])
            congress["committee_count"] = len(committee_map[str_congress])
            congress["max_coeff"] = max_coeff(committee_map[str_congress])
        else:
            congress["committee_count"] = 0
            congress["committee_rank_recips"] = 0

In [ ]:
list(leg_map.values())[0]["congress_map"]

In [ ]:
update_all(engine, tables["leg"], leg_map, verbose = True, increment = 100)